

# RNN for Time Series

RNNs are used for sequence modeling. This tutorial will look at a time series data to be modeled and predicted using RNNs. 

In [1]:
#
# Import Libraries
#

import pandas as pd
import numpy as np

# %matplotlib_inline
import matplotlib.pyplot as plt


## Data

We will use retail data for time-series modeling. 

Link to the dataset:
 https://fred.stlouisfed.org/series/MRTSSM448USN

Information about the Advance Monthly Retail Sales Survey can be found on the Census website at:
https://www.census.gov/retail/marts/about_the_surveys.html

Release: Advance Monthly Sales for Retail and Food Services  
Units:  Millions of Dollars, Not Seasonally Adjusted
Frequency:  Monthly

Suggested Citation:
U.S. Census Bureau, Advance Retail Sales: Clothing and Clothing Accessory Stores [RSCCASN], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/RSCCASN, November 16, 2019.

https://fred.stlouisfed.org/series/RSCCASN

### Read data first -  Use index_col = 'DATE' and 'parse_dates = True' as a parameter.

In [2]:
# Your code to read data

df = pd.read_csv("MRTSSM448USN.csv", index_col = 'DATE', parse_dates = True)

# Print first few rows of data
df.head()

,MRTSSM448USN
DATE,
1992-01-01,6938.0
1992-02-01,7524.0
1992-03-01,8475.0
1992-04-01,9401.0
1992-05-01,9558.0


Does the sales column has any name?

If no, set the name of the colum as 'Sales'.

In [3]:
# Set name of column as 'Sales'. Use - df.columns 
df.columns = ['Sales']

df.head()

,Sales
DATE,
1992-01-01,6938.0
1992-02-01,7524.0
1992-03-01,8475.0
1992-04-01,9401.0
1992-05-01,9558.0


Plot your data - Year vs Sales

<Axes: xlabel='DATE'>

In [4]:
# Your code to plot Year vs Sales. Use either matplot library of pandas dataframe.
df.plot(figsize = (12, 8))

### Next we will do Train Test Split. 

We will use last 1.5 year (18 month) samples for testing. Rest is for training.

In [5]:
# Assign variable test size = 18
# Store length of data in variable length.
# store training size in varialable train_size (Remember training size = total size - test_size)

# from sklearn.model_selection import train_test_split

test_size = 18
data_length = len(df)
data_length

386

In [6]:
len(df) - 18

368

In [7]:
train_size = data_length - test_size
train_size

368

Now, we will find the indexes of the test data. Remember, these are the last 18 indexes in the pandas dataframe.

In [8]:
#Assign the start of test index in data frame to variable test_index.  Remember, it is equal to the length of dataframe - test size

test_index = train_size

In [9]:
# Store all data from 0 to test_index in variable train. Hint - Use df.iloc.
# Store everthing from test_index to the last sample in test variable. Hint - Use df.iloc.

train = df.iloc[:test_index]
test = df.iloc[test_index:]

In [10]:
# Print the size of the train data
# Print the size of the test data
# Print the train data
# Print the test data

print(train.size)
print(test.size)
print(train)
print(test)


368
18
              Sales
DATE               
1992-01-01   6938.0
1992-02-01   7524.0
1992-03-01   8475.0
1992-04-01   9401.0
1992-05-01   9558.0
...             ...
2022-04-01  25224.0
2022-05-01  26156.0
2022-06-01  24338.0
2022-07-01  24530.0
2022-08-01  25651.0

[368 rows x 1 columns]
              Sales
DATE               
2022-09-01  23152.0
2022-10-01  24243.0
2022-11-01  28045.0
2022-12-01  39263.0
2023-01-01  19118.0
2023-02-01  20666.0
2023-03-01  24510.0
2023-04-01  24232.0
2023-05-01  26096.0
2023-06-01  24364.0
2023-07-01  24911.0
2023-08-01  26657.0
2023-09-01  23332.0
2023-10-01  23933.0
2023-11-01  28580.0
2023-12-01  40684.0
2024-01-01  19188.0
2024-02-01  21964.0


### In Neural Networks, we need to Scale Data between 0-1

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()
scaler.fit(train)


MinMaxScaler()

In [13]:
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [14]:
#
# Check if the data has been scaled properly
#

print(scaled_train.max())
print(scaled_test.max())
print(scaled_train.min())
print(scaled_test.min())

1.0
1.0553158319645888
0.0
0.4549427911249687


# Time Series Generator

This class takes in a sequence of data-points gathered at
equal intervals, along with time series parameters such as
stride, length of history, etc., to produce batches for
training/validation.

#### Arguments
    data: Indexable generator (such as list or Numpy array)
        containing consecutive data points (timesteps).
        The data should be at 2D, and axis 0 is expected
        to be the time dimension.
    targets: Targets corresponding to timesteps in `data`.
        It should have same length as `data`.
    length: Length of the output sequences (in number of timesteps).
    sampling_rate: Period between successive individual timesteps
        within sequences. For rate `r`, timesteps
        `data[i]`, `data[i-r]`, ... `data[i - length]`
        are used for create a sample sequence.
    stride: Period between successive output sequences.
        For stride `s`, consecutive output samples would
        be centered around `data[i]`, `data[i+s]`, `data[i+2*s]`, etc.
    start_index: Data points earlier than `start_index` will not be used
        in the output sequences. This is useful to reserve part of the
        data for test or validation.
    end_index: Data points later than `end_index` will not be used
        in the output sequences. This is useful to reserve part of the
        data for test or validation.
    shuffle: Whether to shuffle output samples,
        or instead draw them in chronological order.
    reverse: Boolean: if `true`, timesteps in each output sample will be
        in reverse chronological order.
    batch_size: Number of timeseries samples in each batch
        (except maybe the last one).

# We will use 12 months as input and then predict the next month out


In [15]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
length = 12
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=1)

In [16]:
X, y = generator[0]

print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

Given the Array: 
[0.11586537 0.13217895 0.15865371 0.1844325  0.18880321 0.17833579
 0.17613652 0.21538933 0.18922079 0.20817906 0.23415272 0.43481529]
Predict this y: 
 [[0.13156649]]


### Create the Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU

import tensorflow as tf

In [18]:
# We're only using one feature in our time series
n_features = 1

# Define your own models. 

Use 1. SimpleRNN, LSTM, or GRU neural network.

APIs:
https://keras.io/api/layers/recurrent_layers/

In [19]:
# 
# Your code to create your own model
#

model = Sequential()
model.add(LSTM(10, activation='relu', input_shape=(length, n_features,)))
model.add(Dense(1))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10)                480       
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 491 (1.92 KB)
Trainable params: 491 (1.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.compile(optimizer='adam', loss='mse')

### EarlyStopping and creating a Validation Generator

NOTE: The scaled_test dataset size MUST be greater than your length chosen for your batches. Review video for more info on this.

In [22]:
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=1)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

# Your code to create an object early-stop.

early_stop = EarlyStopping(monitor='val_loss', patience=3)

Now, fit your model.

In [24]:
# Your code to fit your model.
history = model.fit(generator, epochs = 10, validation_data = validation_generator,
                    callbacks = [early_stop])

Epoch 1/10

356/356 [==============================] - 3s 3ms/step - loss: 0.0191 - val_loss: 0.0443
Epoch 2/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0142 - val_loss: 0.0498
Epoch 3/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0149 - val_loss: 0.0438
Epoch 4/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0142 - val_loss: 0.0416
Epoch 5/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0134 - val_loss: 0.0391
Epoch 6/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0139 - val_loss: 0.0361
Epoch 7/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0126 - val_loss: 0.0353
Epoch 8/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0123 - val_loss: 0.0421
Epoch 9/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0115 - val_loss: 0.0291
Epoch 10/10
356/356 [==============================] - 1s 3ms/step - loss: 0.0109 - val_loss: 0.027

In [25]:
# Get Losses from dataframe (hint - model.history.history)- See previous week tutorial.
# Plot losses in the dataframe.

losses = pd.DataFrame(model.history.history)

plt.plot(history.history['loss'], label = 'Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy vs Epochs')
plt.legend()
plt.show()

## Evaluate on Test Data

In [26]:
first_eval_batch = scaled_train[-length:]

In [27]:
n_input = 12
first_eval_batch = first_eval_batch.reshape((1, n_input, n_features))
model.predict(first_eval_batch)


1/1 [==============================] - 0s 160ms/step


array([[0.6676077]], dtype=float32)

In [28]:
# compare with the true result:
scaled_test[0]

array([0.56724479])

#### Try predicting the series!

In [29]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for 
    # grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

1/1 [==============================] - 0s 17ms/step


## Inverse Transformations and Compare

In [30]:
true_predictions = scaler.inverse_transform(test_predictions)

In [31]:
# IGNORE WARNINGS
test['Predictions'] = true_predictions

C:\Users\DEATH\AppData\Local\Temp\ipykernel_20644\1339453577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Predictions'] = true_predictions


# Check and plot predictions

In [32]:
# Print the test variable.

print(test_predictions)

[array([0.6676077], dtype=float32), array([0.6926991], dtype=float32), array([0.7118332], dtype=float32), array([0.700887], dtype=float32), array([0.63228124], dtype=float32), array([0.6649223], dtype=float32), array([0.69038373], dtype=float32), array([0.69275445], dtype=float32), array([0.6969913], dtype=float32), array([0.70563835], dtype=float32), array([0.7189236], dtype=float32), array([0.73212683], dtype=float32), array([0.7443273], dtype=float32), array([0.7510314], dtype=float32), array([0.74975044], dtype=float32), array([0.7500566], dtype=float32), array([0.7628205], dtype=float32), array([0.7757828], dtype=float32)]


In [33]:
# Your code to plot actual sales and predictions.



# Retrain and Forecasting

In [34]:
full_scaler = MinMaxScaler()
scaled_full_data = full_scaler.fit_transform(df)

In [35]:
length = 12 # Length of the output sequences (in number of timesteps)
generator = TimeseriesGenerator(scaled_full_data, 
                                scaled_full_data, length=length, batch_size=1)

In [36]:
#
# Use any neural network model based on RNN
#
# Create the model


In [37]:
# compile the model

In [38]:
# use early_stop

In [39]:
# fit the model

In [40]:
forecast = []
# Replace periods with whatever forecast length you want
periods = 12

first_eval_batch = scaled_full_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(periods):
    
    # get prediction 1 time stamp ahead ([0] is for  grabbing just the number instead of [array])
    
    
    # store prediction forecast.append(current_pred) 
    
    # update batch to now include prediction and drop first value


SyntaxError: incomplete input (3653545150.py, line 15)

In [ ]:
forecast = scaler.inverse_transform(forecast)

### Creating new timestamp index with pandas.

In [ ]:
df

In [ ]:
forecast_index = pd.date_range(start='2024-01-01',periods=periods,freq='MS')

In [ ]:
forecast_df = pd.DataFrame(data=forecast,index=forecast_index,
                           columns=['Forecast'])

In [ ]:
forecast_df

In [ ]:
# Plot sales - Values in dataframe

In [ ]:
# Plot forecast - Values in forecast_df

### Joining pandas plots

https://stackoverflow.com/questions/13872533/plot-different-dataframes-in-the-same-figure

In [ ]:
ax = df.plot()
forecast_df.plot(ax=ax)

In [ ]:
ax = df.plot()
forecast_df.plot(ax=ax)
plt.xlim('2022-01-01','2025-01-01')

# Try the same example with a LSTM and GRU! 
Hint: Use LSTM instead of SimpleRNN!